In [6]:
import numpy as np
import matplotlib
%matplotlib qt 
import matplotlib.pyplot as plt

def coeff(Y):
    """
    Parameters
    ----------
    Y : (n,2) numpy array
    
    Returns
    -------
    coeffs : (2n-2, 4) numpy array
        First n-1 rows  = [a,b,c,d]
        Last  n-1 rows  = [a,b,c,d]
    """
    Y = np.asarray(Y)
    x, y = Y[:, 0], Y[:, 1]
    n = x.shape[0]

    # Build A 
    main = np.full(n - 2, 4.0)
    off  = np.full(n - 3, 1.0)
    A = np.diag(main) + np.diag(off, 1) + np.diag(off, -1)

    # Build z
    z_x = 6 * (x[2:] - 2 * x[1:-1] + x[:-2])
    z_y = 6 * (y[2:] - 2 * y[1:-1] + y[:-2])
    Z = np.column_stack([z_x, z_y])

    # Solve Ac = z
    C_int = np.linalg.solve(A, Z)
    c_x = np.zeros(n)
    c_y = np.zeros(n)
    c_x[1:-1] = C_int[:, 0]
    c_y[1:-1] = C_int[:, 1]

    # Compute coefficients a,b,c
    dx, dy = np.diff(x), np.diff(y)
    a_x, a_y = x[:-1], y[:-1]
    b_x = dx - (2 * c_x[:-1] + c_x[1:]) / 6
    b_y = dy - (2 * c_y[:-1] + c_y[1:]) / 6
    c_x_coef, c_y_coef = c_x[:-1] / 2, c_y[:-1] / 2
    d_x = (c_x[1:] - c_x[:-1]) / 6
    d_y = (c_y[1:] - c_y[:-1]) / 6

    coeff_x = np.column_stack([a_x, b_x, c_x_coef, d_x])
    coeff_y = np.column_stack([a_y, b_y, c_y_coef, d_y])
    return np.vstack([coeff_x, coeff_y])

In [7]:
def plot_all(Y):
    """
    Parameters
    ----------
    Y : (n,2) array
    
    Returns
    -------
    X, Yc : each (100, n-1) array
    """
    # 1) coefficients
    C = coeff(Y)
    n = Y.shape[0]
    m = n - 1

    Cx = C[:m, :]
    Cy = C[m:, :]

    # 2)
    t = np.linspace(0.0, 1.0, 100)
    T = np.column_stack([
        np.ones_like(t),
        t,
        t**2,
        t**3
    ])

    # 3)
    X = T @ Cx.T
    Yc = T @ Cy.T
    return X, Yc

In [8]:
def make_spline(n,color='r'):
    """Prompts user to generate n points.  Plots the cubic spline
    connecting those points.

    Parameters
    ----------
    n : int
    color : string--optional input defaults to red

    Returns
    -------
    None, but it produces a plot
    """
    plt.figure()   #opens an empty figure
    plt.xlim(0, 1) #Restricts the x-axis to [0,1]
    plt.ylim(0, 1) #Restricts the y-axis to [0,1]
    
    pts = np.asarray(plt.ginput(n, timeout = 0)) # it kept timing out on me so I set it to never timeout
    
    X,Y = plot_all(pts)
    plt.plot(X,Y,color)
    return

In [ ]:
make_spline(20, 'green')
plt.axis('equal')
plt.title("Spline 1")